In [1]:
import configparser
import numpy as np

# Testing the things which we need to modify the run_routine.py in order to add the feature.

In [2]:
# Read the configuration file
config = configparser.ConfigParser()
config.read('/Users/amirr/Desktop/Flame/FlameFitSimple/analyses/wimp_sensitivity/inference_configs/mass_mu_test.ini')
print("Sections found:", config.sections())
# Checks out

n_mu = int(config['inference_parameters']['n_mu'])
print("The number of mu's to scan over ",n_mu)

Sections found: ['signal_sources', 'mass_to_mu_ranges', 'inference_parameters']
The number of mu's to scan over  40


In [3]:
# Extract mass-to-mu mappings from the config file
mass_to_mu_ranges = []
for key, value in config['mass_to_mu_ranges'].items():
    values = value.split('#')[0].strip()  # Ignore comments
    mass_min, mass_max, mu_min, mu_max = map(float, values.split(','))
    mass_to_mu_ranges.append((mass_min, mass_max, mu_min, mu_max))

# Print the parsed mass-to-mu ranges
print("Mass-to-mu ranges:")
for range_tuple in mass_to_mu_ranges:
    print(range_tuple)

Mass-to-mu ranges:
(0.0, 0.6, 0.1, 20.0)
(0.6, 1.0, 1.0, 70.0)
(1.0, 100.0, 0.1, 25.0)


In [4]:
def get_mu_range(wimp_mass):
    for mass_min, mass_max, mu_min, mu_max in mass_to_mu_ranges:
        if mass_min <= wimp_mass < mass_max:
            return mu_min, mu_max
    raise ValueError(f"No valid mu range found for WIMP mass {wimp_mass}") # Just to make sure only masses in the config files are given as input

In [5]:
masses=[]
mus_test_dict = dict()
signal_sources = list(config['signal_sources'].keys())

for signal_source in signal_sources:
    # Extract mass from signal_source name (e.g., 'WIMP009' -> 9) 
    signal_name='wimp'
    mass = float(signal_source.replace(signal_name, '')) / 10  # Divide by 10 to get the actual mass.
    masses.append(mass)
    masses.sort()
    # Get the appropriate mu range for this mass
    mu_min, mu_max = get_mu_range(mass)
    mus_test_dict[signal_source] = (mu_min, mu_max)
print("Extracted masses:", masses)

#mu_min, mu_max = get_mu_range(20)
#print(mu_min, mu_max)  # Expected Output: 0.1, 25
# checks out 
#mu_min, mu_max = get_mu_range(100000)
#print(mu_min, mu_max)  # Expected Output: value error 
# checks out

print("The keys of the dict containing the mus for specific masses", mus_test_dict.keys())
print("For example for a WIMP of mass 0.2Gev, the range of mus that will be scanned over are", mus_test_dict["wimp002"])


Extracted masses: [0.2, 0.5, 1.9, 2.0, 20.0]
The keys of the dict containing the mus for specific masses dict_keys(['wimp002', 'wimp005', 'wimp019', 'wimp020', 'wimp0200'])
For example for a WIMP of mass 0.2Gev, the range of mus that will be scanned over are (0.1, 20.0)


In [7]:
mus_test_dict.items()

dict_items([('wimp002', (0.1, 20.0)), ('wimp005', (0.1, 20.0)), ('wimp019', (0.1, 25.0)), ('wimp020', (0.1, 25.0)), ('wimp0200', (0.1, 25.0))])

In [14]:
mus_test_dict_1 = dict()
for signal_source, (mu_min, mu_max) in mus_test_dict.items() :
    print(f"Processing signal source '{signal_source}' with mu range [{mu_min}, {mu_max}]")
    mus_test_dict_1[signal_source] = np.geomspace(mu_min, mu_max, n_mu)
print(mus_test_dict_1)

Processing signal source 'wimp002' with mu range [0.1, 20.0]
Processing signal source 'wimp005' with mu range [0.1, 20.0]
Processing signal source 'wimp019' with mu range [0.1, 25.0]
Processing signal source 'wimp020' with mu range [0.1, 25.0]
Processing signal source 'wimp0200' with mu range [0.1, 25.0]
{'wimp002': array([ 0.1       ,  0.1145515 ,  0.13122045,  0.150315  ,  0.17218808,
        0.19724402,  0.22594598,  0.2588245 ,  0.29648734,  0.33963069,
        0.38905203,  0.44566493,  0.51051585,  0.58480355,  0.66990122,
        0.76738187,  0.87904742,  1.00696198,  1.15349003,  1.32134009,
        1.51361486,  1.73386848,  1.9861723 ,  2.2751901 ,  2.60626432,
        2.9855148 ,  3.41995189,  3.91760609,  4.48767642,  5.14070052,
        5.88874941,  6.7456506 ,  7.72724375,  8.85167339, 10.13972438,
       11.61520607, 13.30539243, 15.24152622, 17.45939645, 20.        ]), 'wimp005': array([ 0.1       ,  0.1145515 ,  0.13122045,  0.150315  ,  0.17218808,
        0.19724402,  

In [23]:
# To cross check if the above range is indeed correct 
mu_min, mu_max = get_mu_range(0.2)
print(np.geomspace(mu_min,mu_max,n_mu))

[ 0.1         0.1145515   0.13122045  0.150315    0.17218808  0.19724402
  0.22594598  0.2588245   0.29648734  0.33963069  0.38905203  0.44566493
  0.51051585  0.58480355  0.66990122  0.76738187  0.87904742  1.00696198
  1.15349003  1.32134009  1.51361486  1.73386848  1.9861723   2.2751901
  2.60626432  2.9855148   3.41995189  3.91760609  4.48767642  5.14070052
  5.88874941  6.7456506   7.72724375  8.85167339 10.13972438 11.61520607
 13.30539243 15.24152622 17.45939645 20.        ]


Okay, now after the code remains OG code remoans the same, jst that everthing will come into a FOR loop ...

In [24]:
# Changes to the run_routine.py to account for the changes we just made in the function.....actually it makes sense to make the change in the external script and run the exisiting function multiple times, however you will have to change the naming of the pkl files as now it would generate a pkl file everytime it does it for one set of ranges of masses ...how will you "stich" all that is the question ...

This itself is enough, all you need to do is feed in different mu_min and mu_max for different masses , the np.geomspscae is already hapening in the OG function, just make a change in the run_routine script and out the functions run_routine in a loop that's all  

However, this would create a different set of pkl files for every mass, so you would need to "pre-stich" the results before it goes into the actual stitching ! (do it tomrrow)  

Apart from this you will also have to make a change in the original config file as now you have additional parameter mass_to_mu_ranges

In [27]:
# Extract signal sources from the configuration
signal_sources = list(config['signal_sources'].keys())  # Partition starts here
print(signal_sources)

# Extract mass-to-mu mappings from the config file
mass_to_mu_ranges = []
for key, value in config['mass_to_mu_ranges'].items():
    values = value.split('#')[0].strip()  # Ignore comments
    mass_min, mass_max, mu_min, mu_max = map(float, values.split(','))
    mass_to_mu_ranges.append((mass_min, mass_max, mu_min, mu_max))

# Print the parsed mass-to-mu ranges
#print("Mass-to-mu ranges:")
#for range_tuple in mass_to_mu_ranges:
#    print(range_tuple)

# Function to get the appropriate mu range for a WIMP mass
def get_mu_range(wimp_mass):
    for mass_min, mass_max, mu_min, mu_max in mass_to_mu_ranges:
        if mass_min <= wimp_mass < mass_max:
            return mu_min, mu_max
    raise ValueError(f"No valid mu range found for WIMP mass {wimp_mass}") # Just to make sure only masses in the config files are given as input

# Validate WIMP masses and prepare mu parameters
mus_test_dict= dict()
masses=[]
for signal_source in signal_sources:
    # Extract WIMP mass (e.g., 'WIMP009' -> 0.9 GeV)
    mass = int(''.join(filter(str.isdigit, signal_source))) / 10
    masses.append(mass) 
    # Get the mu range for the mass
    mu_min, mu_max= get_mu_range(mass)
    mus_test_dict[signal_source] = (mu_min, mu_max)
print("Extracted masses:", masses)
# Extract other inference parameters
n_mu = int(config['inference_parameters']['n_mu'])
ntoys = int(config['inference_parameters']['ntoys'])

# Initialize the inference helper
inference_helper = InferenceHelper(likelihood_container, background_sources, signal_sources)

# Invoke the run routine with dynamic mu ranges
for signal_source, (mu_min, mu_max) in mus_test_dict.items() :
    print(f"Processing signal source '{signal_source}' with mu range [{mu_min}, {mu_max}]")
    inference_helper.run_routine(
        num_toys=ntoys, 
        output_dir=args.output,
        mu_min=mu_min, 
        mu_max=mu_max, 
        n_mu=n_mu
    )


['wimp002', 'wimp005', 'wimp019', 'wimp020', 'wimp0200']
Mass-to-mu ranges:
(0.0, 0.6, 0.1, 20.0)
(0.6, 1.0, 1.0, 70.0)
(1.0, 100.0, 0.1, 25.0)
Extracted masses: [0.2, 0.5, 1.9, 2.0, 20.0]


NameError: name 'InferenceHelper' is not defined